<a href="https://colab.research.google.com/github/rubensdmp/GeoUy/blob/main/GetLoc%26LinksCampismo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Script para extraer información geográfica de fotografías**

## **Resumen**

Este pequeño script extrae la información ubicación geográfica de fotografías y luego se crea un CSV con dicha ubicación y el link de google drive para acceder públicamente a las fotos.
Este CSV luego es cargado en R y a traves de la biblioteca MapView se plotean como popups las fotos en su ubicación original.

Desarrollado por [**Rubens Merlino**](https://www.linkedin.com/in/rubens-merlino-uy/)


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [20]:
from pydrive.drive import GoogleDrive

drive = GoogleDrive(gauth) # Create GoogleDrive instance with authenticated GoogleAuth instance

folder_id = '18nDmE8ttKu9jC-zogZ1uyc5knRstxdtf'

str = "\'" + folder_id + "\'" + " in parents and trashed=false"


# Auto-iterate through all files in the root folder.
file_list = drive.ListFile({'q': str}).GetList()


In [10]:
import os
import glob
import pandas as pd
import requests
!pip install exifread
import exifread

from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
path = '/content/drive/MyDrive/07. Personal/11. Travesías & Campismo/FotosMapa/'
fms=[]
names=[]
lats=[]
lons=[]
caps=[]
rutas=[]

#for file1 in file_list:
#for file1 in glob.glob(path):
    

for file1 in file_list:
    fms.append(file1['title'])
    rutas.append('https://drive.google.com/uc?id=' + file1['id'])
  
    with open(path + file1['title'], 'rb') as f:
        tags = exifread.process_file(f)
        lat_ref = tags["GPS GPSLatitudeRef"].printable
        lat = tags["GPS GPSLatitude"].printable[1:-1].replace(" ", "").replace("/", ",").split(",")
        lat = float(lat[0]) + float(lat[1]) / 60 + float(lat[2]) / float(lat[3]) / 3600
        
        if lat_ref != "N":
            lat = lat * (-1)
        lon_ref = tags["GPS GPSLongitudeRef"].printable
        lon = tags["GPS GPSLongitude"].printable[1:-1].replace(" ", "").replace("/", ",").split(",")
        lon = float(lon[0]) + float(lon[1]) / 60 + float(lon[2]) / float(lon[3]) / 3600
        
        if lon_ref != "E":
            lon = lon * (-1)
        
        names.append(f.name[60:])        
        lats.append(lat)
        lons.append(lon)
        caps.append(tags["EXIF DateTimeOriginal"])

  

In [22]:
#defino cabezales del CSV creado como un dataframe
#nombre foto / Ubicación

df = pd.DataFrame()
df['formacion'] = fms
#df['name'] = names
df['lat'] = lats
df['lon'] = lons
df['captura'] = caps
df['ruta'] = rutas

#df = df.assign(Edad=None)

df.to_csv('/content/drive/My Drive/07. Personal/11. Travesías & Campismo/FotosMapa/file.csv')

#files.download("fms.csv")

df.head()

,formacion,lat,lon,captura,ruta
0,2020-01-03 10.40.38.jpg,-33.779231,-54.656614,2020:01:03 10:40:38,https://drive.google.com/uc?id=1sx0EhPkgaA7WWG...
1,2020-01-03 10.42.32.jpg,-33.779259,-54.656575,2020:01:03 10:42:32,https://drive.google.com/uc?id=1tK795ppRNhx1rd...
2,2020-01-03 22.24.53.jpg,-33.765682,-54.541686,2020:01:03 22:24:53,https://drive.google.com/uc?id=1pDEsYh9ftI88-e...
3,2020-01-03 15.33.33.jpg,-33.773281,-54.606093,2020:01:03 15:33:33,https://drive.google.com/uc?id=1pit2jWcpSkekxX...
4,2020-01-03 10.30.58.jpg,-33.779538,-54.655445,2020:01:03 10:30:58,https://drive.google.com/uc?id=1s6yBFEocQJe3rO...
